In [47]:
import mysql.connector

# #Connect to DB 
conn=mysql.connector.connect(host="localhost",user="root",password="Cuppuchinno!1",database="testdb")

#Established the connection
cursor= conn.cursor()

print("DB Connected Successfully")

DB Connected Successfully


In [48]:
#Import libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [49]:
import os


downloads_folder = os.path.expanduser('~/Downloads')

# Specify the name of your Excel file
excel_file_name = 'training_retail_supply_chain_data_test.xlsx' 

# Construct the full path to the Excel file
file_path = os.path.join(downloads_folder, excel_file_name)
df = pd.read_excel(file_path)


In [50]:
total_test_cases = 0
total_test_cases_passed = 0
total_test_cases_failed = 0

In [51]:
from datetime import datetime
import os
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
file_name = "Targetdata_validation_file.txt"
filename_base, file_extension = os.path.splitext(file_name)
new_filename = f"{filename_base}_{timestamp}{file_extension}"

In [52]:
def update_Targetdata_validation_file(result, test_case_name):
  global total_test_cases, total_test_cases_passed, total_test_cases_failed,new_filename

  total_test_cases += 1
  data_validation_file_path = os.path.join(downloads_folder, new_filename)
  if len(result) == 0:
    total_test_cases_passed += 1
    # create a text file and append the validation reult
    with open(data_validation_file_path, 'a') as f:
      f.write(f"{total_test_cases} | {test_case_name} validation passed |Pass\n") 
      f.close()
  else:
    total_test_cases_failed += 1
    with open(data_validation_file_path, 'a') as f:
      f.write(f"{total_test_cases} | {test_case_name} validation failed |Fail\n") 
      f.write("Invalid rows:\n")
      f.write(str(result))
      f.write("\n")
      f.close()


In [53]:
def Target_Schema_validation(df):
    cursor.execute("DESCRIBE Retail_Data_New_test")
    test_case_name = 'Target_Schema_validation_check(column name, column count, datatype)'
    result = []
    mysql_schema = cursor.fetchall()

# Process MySQL schema into a dictionary for easier comparison
    mysql_cols = {col[0]: col[1] for col in mysql_schema} # col[0] is name, col[1] is type

   

    # Get DataFrame schema
    df_cols = {col: str(df[col].dtype) for col in df.columns}
    mysql_column_names = set(mysql_cols.keys())
    df_column_names = set(df_cols.keys())

    if mysql_column_names == df_column_names:
        print("Column names match.")
        # Simple type mapping (can be expanded for more complex types)
        type_mapping = {
            'int': ['int64', 'int32'],
            'varchar': ['object'],
            'text': ['object'],
            'float': ['float64', 'float32'],
            'double': ['float64'],
            'datetime': ['datetime64[ns]'],
            'date': ['datetime64[ns]'],
            'tinyint': ['int64', 'bool'] # For boolean representation
        }

        data_type_mismatches = []
        for col_name in mysql_column_names.intersection(df_column_names):
            mysql_type = mysql_cols[col_name].split('(')[0].lower() # Remove size specifiers
            df_type = df_cols[col_name].lower()
        
            if mysql_type not in type_mapping or df_type not in type_mapping[mysql_type]:
                data_type_mismatches.append(f"Column '{col_name}': MySQL type '{mysql_cols[col_name]}' vs DataFrame type '{df_cols[col_name]}'")
        
        if not data_type_mismatches:
            print("Data types are compatible.")
        else:
            result.append(data_type_mismatches)
            print("Data type mismatches found:")
            for mismatch in data_type_mismatches:
                print(mismatch)
        
    else:
        print("Column names do not match.")
        
    
    update_Targetdata_validation_file(result, test_case_name)
    return(result)

In [54]:
Target_Schema_validation(df)

Column names match.
Data types are compatible.


[]

In [55]:
def Target_Nulls_check():
    cursor.execute("DESCRIBE Retail_Data_New_test")
    test_case_name = 'Target_Nulls_check'
    
    result = []
    

# Process MySQL schema into a dictionary for easier comparison
    mysql_cols = [col[0] for col in cursor.fetchall()] # col[0] is name, col[1] is type
    table_name = "Retail_Data_New_test"
    columns_to_check = [col for col in mysql_cols] # Replace with your actual column names
    
    where_clauses = [f"{col} IS NULL" for col in columns_to_check]
    sql_query = f"SELECT * FROM {table_name} WHERE {' OR '.join(where_clauses)};"
   

    cursor.execute(sql_query)
    null_rows = cursor.fetchall()
  
    if not null_rows :
        print('there are no nulls')
    else:
        result= [null_rows]
   
    
    update_Targetdata_validation_file(result, test_case_name)
    return(result)

In [56]:
Target_Nulls_check()

there are no nulls


[]

In [57]:
def Date_format_check():
    test_case_name = 'Date_format_check'
    result = []
    incorrect_format_query = f"""
    SELECT *
    FROM Retail_Data_New_test
    WHERE STR_TO_DATE(Date, '%Y-%m-%d %H:%i:%s') IS NULL
    OR DATE_FORMAT(STR_TO_DATE(Date, '%Y-%m-%d %H:%i:%s'), '%Y-%m-%d %H:%i:%s') != Date;
    """
    cursor.execute(incorrect_format_query)
    incorrect_format_rows = cursor.fetchall()

    if not incorrect_format_rows:
        print("\nAll rows have correct date format")
         
    else:
        result.append(incorrect_format_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result)


In [58]:
Date_format_check()


All rows have correct date format


[]

In [59]:
def check_unique_product_id(): 
    test_case_name = 'check_unique_product_id'
    result = []
    incorrect_data_query = f"""
    select Date, Product_ID, count(*) from retail_data_new_test 
    group by 1,2
    having count(*) > 1
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result) 


In [60]:
check_unique_product_id()


All rows have correct data


[]

In [61]:
def invalid_values_units_sold(): 
    test_case_name = 'invalid_values_units_sold'
    result = []
    incorrect_data_query = f"""
    select * from retail_data_new_test 
    where Units_Sold <= 0;
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result) 

In [62]:
invalid_values_units_sold() 


All rows have correct data


[]

In [63]:
def invalid_values_revenue(): 
    test_case_name = 'invalid_values_revenue'
    result = []
    incorrect_data_query = f"""
    select * from retail_data_new_test 
    where Revenue <= 0;
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result)

In [64]:
invalid_values_revenue()


All rows have correct data


[]

In [65]:
def invalid_threshold_levels():
    test_case_name = 'invalid_threshold_levels'
    result = []
    incorrect_data_query = f"""
    select * from retail_data_new_test 
    where Stock_Level <= Reorder_Threshold;
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result)


In [66]:
invalid_threshold_levels()


[[(datetime.datetime(2023, 1, 1, 0, 0),
   199,
   'Product E',
   'Home',
   24,
   3930,
   15,
   18,
   287,
   3,
   20,
   0)]]

In [67]:
def null_values_supplier_id():
    test_case_name = 'null_values_supplier_id'
    result = []
    incorrect_data_query = f"""
    select * from retail_data_new_test 
    where Supplier_ID is null;
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result)


In [68]:
null_values_supplier_id()



All rows have correct data


[]

In [69]:
def discount_nonNumeric_values():
    test_case_name = 'discount_nonNumeric_values'
    result = []
    incorrect_data_query = f"""
    select * from retail_data_new_test 
    where Discount_Applied REGEXP '[^0-9.]';
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result)


In [70]:
discount_nonNumeric_values()  



All rows have correct data


[]

In [71]:
def ReturnFlag_boolvalueCheck():
    test_case_name = 'ReturnFlag_boolvalueCheck'
    result = []
    incorrect_data_query = f"""
    select * from retail_data_new_test 
    where Return_Flag not in (0,1);
    """
    cursor.execute(incorrect_data_query)
    incorrect_rows = cursor.fetchall()

    if not incorrect_rows:
        print("\nAll rows have correct data")
         
    else:
        result.append(incorrect_rows)
    update_Targetdata_validation_file(result, test_case_name)
    return(result)


In [72]:
ReturnFlag_boolvalueCheck() 



All rows have correct data


[]

In [73]:
import os
import glob
import pandas as pd

# Folder path where your text files are stored
folder_path = downloads_folder

file_name = "Sourcedata_validation_file_20251027_140223.txt"   # example: "data_2025_10_27.txt"

# Combine path and file name
file_path = os.path.join(folder_path, file_name)
df1 = pd.read_csv(file_path, sep="|", encoding="utf-8",header = None)
df1

0                                              1     2
0  1            Date format check validation passed  Pass
1  2      check_unique_product_id validation passed  Pass
2  3    invalid_values_units_sold validation passed  Pass
3  4       invalid_values_revenue validation passed  Pass
4  5     invalid_threshold_levels validation failed  Fail
5  6      null_values_supplier_id validation passed  Pass
6  7   discount_nonNumeric_values validation passed  Pass
7  8    ReturnFlag_boolvalueCheck validation passed  Pass

In [74]:
file_name = "Targetdata_validation_file_20251027_141632.txt"   # example: "data_2025_10_27.txt"

# Combine path and file name
file_path = os.path.join(folder_path, file_name)
df2 = pd.read_csv(file_path, sep="|", encoding="utf-8",header = None)
df2


0                                                  1     2
0   1   Target_Schema_validation_check(column name, c...  Pass
1   2              Target_Nulls_check validation passed   Pass
2   3               Date_format_check validation passed   Pass
3   4         check_unique_product_id validation passed   Pass
4   5       invalid_values_units_sold validation passed   Pass
5   6          invalid_values_revenue validation passed   Pass
6   7        invalid_threshold_levels validation failed   Fail
7   8         null_values_supplier_id validation passed   Pass
8   9      discount_nonNumeric_values validation passed   Pass
9  10       ReturnFlag_boolvalueCheck validation passed   Pass

In [75]:
def build_validation(df1,df2):
    res = 'Build Pass'
    for elem in df1:
        if elem == 'Fail':
            return('Build failed')
    for elem in df2:
        if elem == 'Fail':
            return('Build failed')       
    return(ans)    

In [76]:
build_validation(df1.iloc[:, -1],df2.iloc[:, -1])

'Build failed'

In [77]:
def close_db_connection():
        cursor.close()
        conn.close()
        print("MySQL connection closed.")